In [1]:
import cv2
import numpy as np
from scipy.spatial import distance
from imutils import face_utils
import dlib

# Load pre-trained face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Function to compute eye aspect ratio (EAR)
def eye_aspect_ratio(eye):
    # Compute the euclidean distances between the two sets of vertical eye landmarks
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    # Compute the euclidean distance between the horizontal eye landmark
    C = distance.euclidean(eye[0], eye[3])
    # Compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    return ear

# Load your machine learning model for eye state classification here

# Load the video stream
cap = cv2.VideoCapture(0) # Change 0 to the video file path if using a file

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the grayscale frame
    faces = detector(gray)
    
    for face in faces:
        # Determine facial landmarks
        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)
        
        # Extract the eye regions from facial landmarks
        left_eye = shape[42:48]
        right_eye = shape[36:42]
        
        # Compute eye aspect ratios
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        
        # Average the eye aspect ratios for both eyes
        ear = (left_ear + right_ear) / 2.0
        
        # Perform eye state classification using your machine learning model
        # Example:
        # eye_state = model.predict(ear)
        
        # Draw the eye regions and ear on the frame
        cv2.drawContours(frame, [cv2.convexHull(left_eye)], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [cv2.convexHull(right_eye)], -1, (0, 255, 0), 1)
        cv2.putText(frame, f'EAR: {ear}', (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # If eye state is closed, perform drowsiness detection action
        # Example:
        # if eye_state == 'closed':
        #     perform_drowsiness_detection_action()
    
    # Display the frame
    cv2.imshow("Frame", frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video stream and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'dlib'